In [8]:
import tkinter as tk
import random
import time

ASDF_KEYS = ["a", "s", "d", "f"]

# 難度：等待時間範圍 + 反應超時限制（秒）
DIFFICULTY = {
    "Easy":   {"wait_min": 1.8, "wait_max": 4.0, "timeout": 2.0},
    "Normal": {"wait_min": 1.2, "wait_max": 3.2, "timeout": 1.4},
    "Hard":   {"wait_min": 0.8, "wait_max": 2.4, "timeout": 0.5},
}

# 模式：單鍵（空白鍵）/ ASDF
MODE = {
    "Single Key (Space)": "single",
    "ASDF": "asdf",
}


class ReactionRushApp:
    def __init__(self, root: tk.Tk):
        self.root = root
        self.root.title("Reaction Rush - 反應力遊戲 (Modes + Difficulty + Combo + Trap)")
        self.root.geometry("600x500")
        self.root.resizable(False, False)

        # ----- Game state -----
        self.total_rounds = 10
        self.round_idx = 0

        self.waiting = False      # 正在隨機等待提示出現（此時按鍵算 false start）
        self.accepting = False    # 提示已出現，等待玩家輸入
        self.expected_key = None
        self.prompt_time = 0.0

        self.after_id = None
        self.timeout_after_id = None

        # ----- Stats -----
        self.reaction_times = []  # 只記錄「按對」的反應時間(ms)
        self.correct = 0
        self.wrong = 0
        self.false_start = 0
        self.too_slow = 0

        # Combo
        self.combo = 0
        self.max_combo = 0

        # Trap (假提示)：顯示 WAIT，規則是「不要按」
        self.is_trap_round = False
        self.trap_prob = 0.18  # 假提示出現機率（可調 0.10~0.25）

        # ----- Settings -----
        self.mode_var = tk.StringVar(value="ASDF")
        self.diff_var = tk.StringVar(value="Normal")
        self.rounds_var = tk.StringVar(value=str(self.total_rounds))

        # ----- UI -----
        tk.Label(root, text="Reaction Rush", font=("Arial", 22, "bold")).pack(pady=10)

        self.info_lbl = tk.Label(
            root,
            text="玩法：提示出現後按下正確按鍵（Space 或 A/S/D/F）\n"
                 "Trap 回合會顯示 WAIT：這時要忍住不按！\n"
                 "提示出現前亂按算 False Start（Combo 會歸零）",
            font=("Arial", 11),
            justify="center"
        )
        self.info_lbl.pack(pady=4)

        self.center_lbl = tk.Label(
            root,
            text="按 Start 開始",
            font=("Arial", 40, "bold"),
            width=12,
            height=2
        )
        self.center_lbl.pack(pady=16)

        self.status_lbl = tk.Label(root, text="", font=("Arial", 12), justify="left")
        self.status_lbl.pack(pady=6)

        panel = tk.Frame(root)
        panel.pack(pady=8)

        # 回合數
        tk.Label(panel, text="回合數：", font=("Arial", 11)).grid(row=0, column=0, padx=6, pady=4, sticky="e")
        self.rounds_entry = tk.Entry(panel, textvariable=self.rounds_var, width=6, font=("Arial", 11))
        self.rounds_entry.grid(row=0, column=1, padx=6, pady=4, sticky="w")

        # 模式
        tk.Label(panel, text="模式：", font=("Arial", 11)).grid(row=1, column=0, padx=6, pady=4, sticky="e")
        self.mode_menu = tk.OptionMenu(panel, self.mode_var, *MODE.keys())
        self.mode_menu.config(width=20)
        self.mode_menu.grid(row=1, column=1, padx=6, pady=4, sticky="w")

        # 難度
        tk.Label(panel, text="難度：", font=("Arial", 11)).grid(row=2, column=0, padx=6, pady=4, sticky="e")
        self.diff_menu = tk.OptionMenu(panel, self.diff_var, *DIFFICULTY.keys())
        self.diff_menu.config(width=20)
        self.diff_menu.grid(row=2, column=1, padx=6, pady=4, sticky="w")

        # 按鈕
        btns = tk.Frame(panel)
        btns.grid(row=0, column=2, rowspan=3, padx=16)

        self.start_btn = tk.Button(btns, text="Start", width=12, command=self.start_game)
        self.start_btn.pack(pady=4)

        self.reset_btn = tk.Button(btns, text="Reset", width=12, command=self.reset_game)
        self.reset_btn.pack(pady=4)

        # 綁定按鍵
        root.bind("<KeyPress>", self.on_keypress)

        self.update_status("尚未開始")

    # ---------- Helpers ----------
    def set_center(self, msg: str):
        self.center_lbl.config(text=msg)

    def update_status(self, msg: str):
        self.status_lbl.config(text=msg)

    def safe_cancel_after(self):
        if self.after_id is not None:
            try:
                self.root.after_cancel(self.after_id)
            except Exception:
                pass
            self.after_id = None

    def safe_cancel_timeout(self):
        if self.timeout_after_id is not None:
            try:
                self.root.after_cancel(self.timeout_after_id)
            except Exception:
                pass
            self.timeout_after_id = None

    def parse_rounds(self) -> int:
        try:
            n = int(self.rounds_var.get().strip())
            if 1 <= n <= 50:
                return n
        except Exception:
            pass
        return 10

    def current_settings(self):
        diff = self.diff_var.get()
        if diff not in DIFFICULTY:
            diff = "Normal"
        mode_name = self.mode_var.get()
        if mode_name not in MODE:
            mode_name = "ASDF"
        return DIFFICULTY[diff], MODE[mode_name], diff, mode_name

    def lock_controls(self, locked: bool):
        state = "disabled" if locked else "normal"
        self.start_btn.config(state=state if locked else "normal")  # Start 只在遊戲中 disabled
        self.rounds_entry.config(state=state)
        self.mode_menu.config(state=state)
        self.diff_menu.config(state=state)

    # ---------- Game flow ----------
    def start_game(self):
        if self.waiting or self.accepting:
            return

        self.total_rounds = self.parse_rounds()
        self.round_idx = 0

        self.reaction_times.clear()
        self.correct = 0
        self.wrong = 0
        self.false_start = 0
        self.too_slow = 0
        self.combo = 0
        self.max_combo = 0

        self.lock_controls(True)
        self.next_round()

    def reset_game(self):
        self.safe_cancel_after()
        self.safe_cancel_timeout()

        self.waiting = False
        self.accepting = False
        self.expected_key = None
        self.is_trap_round = False

        self.round_idx = 0
        self.reaction_times.clear()
        self.correct = 0
        self.wrong = 0
        self.false_start = 0
        self.too_slow = 0
        self.combo = 0
        self.max_combo = 0

        self.set_center("按 Start 開始")
        self.update_status("已重置")
        self.lock_controls(False)

    def next_round(self):
        if self.round_idx >= self.total_rounds:
            self.finish_game()
            return

        settings, mode, diff_name, mode_name = self.current_settings()

        self.round_idx += 1
        self.accepting = False
        self.waiting = True

        # 這回合是否為 Trap（假提示）
        self.is_trap_round = (random.random() < self.trap_prob)

        # choose expected key based on mode
        if mode == "asdf":
            self.expected_key = random.choice(ASDF_KEYS)
            show_hint = f"模式：ASDF（正常回合要按 {self.expected_key.upper()}）"
        else:
            self.expected_key = "space"
            show_hint = "模式：單鍵（正常回合要按 Space）"

        trap_text = "本回合可能出現 Trap（WAIT：不要按）"
        combo_text = f"Combo：{self.combo}（Max：{self.max_combo}）"

        self.set_center("...")
        self.update_status(
            f"第 {self.round_idx}/{self.total_rounds} 回合\n"
            f"{show_hint}\n"
            f"難度：{diff_name}（超時 {settings['timeout']:.1f}s）\n"
            f"{combo_text}\n"
            f"{trap_text}\n"
            f"提示出現前請別亂按（False Start）"
        )

        # random wait
        wait_s = random.uniform(settings["wait_min"], settings["wait_max"])
        delay_ms = int(wait_s * 1000)

        self.safe_cancel_after()
        self.after_id = self.root.after(delay_ms, self.show_prompt)

    def show_prompt(self):
        settings, mode, diff_name, mode_name = self.current_settings()

        self.waiting = False
        self.accepting = True
        self.prompt_time = time.perf_counter()

        # Trap Round：顯示 WAIT，規則是「不要按」
        if self.is_trap_round:
            self.set_center("WAIT")
            self.update_status(f"Trap 回合！忍住不要按！（{diff_name}，超時 {settings['timeout']:.1f}s）")
        else:
            if mode == "asdf":
                self.set_center(self.expected_key.upper())
                self.update_status(f"按下：{self.expected_key.upper()}（{diff_name}，超時 {settings['timeout']:.1f}s）")
            else:
                self.set_center("GO!")
                self.update_status(f"立刻按 Space！（{diff_name}，超時 {settings['timeout']:.1f}s）")

        # start timeout countdown
        self.safe_cancel_timeout()
        self.timeout_after_id = self.root.after(int(settings["timeout"] * 1000), self.on_timeout)

    def on_timeout(self):
        if not self.accepting:
            return

        self.accepting = False

        # Trap Round：時間到仍沒按鍵 → 成功
        if self.is_trap_round:
            self.correct += 1
            self.combo += 1
            self.max_combo = max(self.max_combo, self.combo)

            self.set_center("NICE")
            self.update_status(f"忍住成功！Combo：{self.combo}（Max：{self.max_combo}）")

            self.safe_cancel_after()
            self.after_id = self.root.after(700, self.next_round)
            return

        # 正常回合：超時 = Too Slow
        self.too_slow += 1
        self.combo = 0

        self.set_center("太慢了")
        self.update_status(f"超時！(Too Slow +1) 目前 {self.too_slow} 次\nCombo 歸零")

        self.safe_cancel_after()
        self.after_id = self.root.after(700, self.next_round)

    def finish_game(self):
        self.safe_cancel_after()
        self.safe_cancel_timeout()
        self.waiting = False
        self.accepting = False

        # 注意：這裡把 too_slow 也算一次嘗試，trap 成功/失敗也包含在 correct/wrong
        total_attempts = self.correct + self.wrong + self.too_slow
        accuracy = (self.correct / total_attempts * 100.0) if total_attempts > 0 else 0.0

        if self.reaction_times:
            avg = sum(self.reaction_times) / len(self.reaction_times)
            best = min(self.reaction_times)
            worst = max(self.reaction_times)
            summary = (
                "=== 結算 ===\n"
                f"回合數：{self.total_rounds}\n"
                f"正確：{self.correct} / 按錯：{self.wrong} / Too Slow：{self.too_slow} / False Start：{self.false_start}\n"
                f"正確率（含超時）：{accuracy:.1f}%\n"
                f"平均反應：{avg:.1f} ms\n"
                f"最快：{best} ms\n"
                f"最慢：{worst} ms\n"
                f"最高連擊：{self.max_combo}"
            )
        else:
            summary = (
                "=== 結算 ===\n"
                f"回合數：{self.total_rounds}\n"
                f"正確：{self.correct} / 按錯：{self.wrong} / Too Slow：{self.too_slow} / False Start：{self.false_start}\n"
                f"正確率（含超時）：{accuracy:.1f}%\n"
                f"最高連擊：{self.max_combo}\n"
                "沒有成功回合，無法計算反應時間。"
            )

        self.set_center("完成！")
        self.update_status(summary)

        self.lock_controls(False)

    # ---------- Input ----------
    def on_keypress(self, event):
        settings, mode, diff_name, mode_name = self.current_settings()
        key = (event.keysym or "").lower()

        # mode-specific valid keys
        if mode == "asdf":
            valid = set(ASDF_KEYS)
        else:
            valid = {"space"}  # keysym for space is "space"

        # 只接受該模式的按鍵
        if key not in valid:
            return

        # 提示出現前按 => false start
        if self.waiting:
            self.false_start += 1
            self.combo = 0

            self.set_center("太早了！")
            self.update_status(
                f"False Start +1（目前 {self.false_start} 次）\n"
                f"Combo 歸零\n"
                f"第 {self.round_idx}/{self.total_rounds} 回合重來"
            )

            self.safe_cancel_after()
            self.safe_cancel_timeout()
            self.after_id = self.root.after(700, self.retry_same_round)
            return

        # 不在接受狀態就忽略
        if not self.accepting:
            return

        # 進入接受狀態：先停止超時計時
        self.accepting = False
        self.safe_cancel_timeout()

        # Trap Round：顯示 WAIT，按了就失敗（不計反應時間）
        if self.is_trap_round:
            self.wrong += 1
            self.combo = 0

            self.set_center("TRAP!")
            self.update_status("你被騙了！這回合要忍住不按\nCombo 歸零")

            self.safe_cancel_after()
            self.after_id = self.root.after(800, self.next_round)
            return

        # 正常回合：計算反應時間
        rt_ms = int((time.perf_counter() - self.prompt_time) * 1000)

        if mode == "asdf":
            is_correct = (key == self.expected_key)
            pressed_show = key.upper()
            expected_show = self.expected_key.upper()
        else:
            is_correct = (key == "space")
            pressed_show = "SPACE"
            expected_show = "SPACE"

        if is_correct:
            self.correct += 1
            self.reaction_times.append(rt_ms)

            self.combo += 1
            self.max_combo = max(self.max_combo, self.combo)

            self.set_center("OK")
            self.update_status(f"正確！反應時間：{rt_ms} ms\nCombo：{self.combo}（Max：{self.max_combo}）")
        else:
            self.wrong += 1
            self.combo = 0

            self.set_center("錯")
            self.update_status(
                f"按錯了！你按 {pressed_show}，正確是 {expected_show}\n"
                f"反應時間：{rt_ms} ms\n"
                "Combo 歸零"
            )

        self.safe_cancel_after()
        self.after_id = self.root.after(800, self.next_round)

    def retry_same_round(self):
        # 不增加 round_idx，重跑等待
        settings, mode, diff_name, mode_name = self.current_settings()

        self.accepting = False
        self.waiting = True

        self.set_center("...")
        self.update_status(f"第 {self.round_idx}/{self.total_rounds} 回合：準備中（別亂按）")

        wait_s = random.uniform(settings["wait_min"], settings["wait_max"])
        delay_ms = int(wait_s * 1000)

        self.safe_cancel_after()
        self.after_id = self.root.after(delay_ms, self.show_prompt)


if __name__ == "__main__":
    root = tk.Tk()
    app = ReactionRushApp(root)
    root.mainloop()
